In [1]:
import pandas as pd
import time
import ray
from oanda_forex_utilities import *
import os
import warnings


In [2]:
account_details = get_oanda_account_credentials("demo_account_1.16")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

print(account_id)
print(access_token)

101-003-18483320-019
87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5


In [9]:

ray.init()
@ray.remote
def automated_trade(currency_pair):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')   
        get_extraction_time("M5")
        time.sleep(1.5)
        state = True
        complete_list = []
        trigger_bar ={}
        while state == True:
            df = get_candle_data(account_id, access_token, currency_pair, "M5", 500)
            df_longer_time = get_candle_data(account_id, access_token, currency_pair, "M30", 500)

            df_with_indicators = get_macd_indicator(df)
            df_with_indicators = get_adx_indicator(df_with_indicators)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 8)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 13)
            df_longer_time_with_indicators = get_ema_indicator(df_with_indicators, window = 21)
            df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

            df_longer_time_with_indicators = get_ema_indicator(df_longer_time, window = 8)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 13)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 21)

            if df_with_indicators_complete["time"].iloc[-1] not in complete_list:
                complete_list.append(df_with_indicators_complete["time"].iloc[-1])

                if len(trigger_bar) == 0:
                    if (
                            (df_with_indicators_complete["price_high"].iloc[-1] >= df_with_indicators_complete["ema_8"].iloc[-1]) and
                            (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["ema_21"].iloc[-1]) and

                            (df_with_indicators_complete["price_high"].iloc[-2] < df_with_indicators_complete["ema_8"].iloc[-2]) and
                            (df_with_indicators_complete["price_high"].iloc[-3] < df_with_indicators_complete["ema_8"].iloc[-3]) and
                            (df_with_indicators_complete["price_high"].iloc[-4] < df_with_indicators_complete["ema_8"].iloc[-4]) and
    #                         (df_with_indicators_complete["price_high"].iloc[-5] < df_with_indicators_complete["ema_8"].iloc[-5]) and

                            (df_longer_time_with_indicators["price_high"].iloc[-1] < df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                            (df_longer_time_with_indicators["price_high"].iloc[-2] < df_longer_time_with_indicators["ema_8"].iloc[-2])


                        ):
                        trigger_price = min(df_with_indicators_complete["price_close"].iloc[-1],
                                            df_with_indicators_complete["price_close"].iloc[-2],
                                            df_with_indicators_complete["price_close"].iloc[-3],
                                            df_with_indicators_complete["price_close"].iloc[-4],
                                            df_with_indicators_complete["price_close"].iloc[-5])
                        trigger_bar = {"SELL": trigger_price}
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " TRIGGER BAR - SELL")
                        print(trigger_bar["SELL"])

                    elif(
                            (df_with_indicators_complete["price_low"].iloc[-1] <= df_with_indicators_complete["ema_8"].iloc[-1]) and
                            (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["ema_21"].iloc[-1]) and

                            (df_with_indicators_complete["price_low"].iloc[-2] > df_with_indicators_complete["ema_8"].iloc[-2]) and
                            (df_with_indicators_complete["price_low"].iloc[-3] > df_with_indicators_complete["ema_8"].iloc[-3]) and
                            (df_with_indicators_complete["price_low"].iloc[-4] > df_with_indicators_complete["ema_8"].iloc[-4]) and
    #                         (df_with_indicators_complete["price_low"].iloc[-5] > df_with_indicators_complete["ema_8"].iloc[-5]) and

                            (df_longer_time_with_indicators["price_low"].iloc[-1] > df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                            (df_longer_time_with_indicators["price_low"].iloc[-2] > df_longer_time_with_indicators["ema_8"].iloc[-2])
                        ):
                        trigger_price = max(df_with_indicators_complete["price_close"].iloc[-1],
                                            df_with_indicators_complete["price_close"].iloc[-2],
                                            df_with_indicators_complete["price_close"].iloc[-3],
                                            df_with_indicators_complete["price_close"].iloc[-4],
                                            df_with_indicators_complete["price_close"].iloc[-5])
                        trigger_bar = {"BUY": trigger_price}
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " TRIGGER BAR - BUY")
                        print(trigger_bar["SELL"])

                    else:
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " NOTHING HAPPENS")
                        trigger_bar = {}
                        pass

                else:
                    print("STATUS: 1 TRIGGER BAR")
                    if (
                            (list(trigger_bar.keys())[0] == "SELL") &
                            (df_with_indicators_complete["price_low"] <= trigger_bar['SELL'])
                       ):
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " SELL ORDER FILLED")
                        if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                            stop_loss = 1000
                        else:
                            stop_loss = 10
                        create_market_order_with_sl_tp(account_id, access_token, currency_pair, "SELL", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5))
                        trigger_bar = {}

                    elif (

                        (list(trigger_bar.keys())[0] == "BUY") &
                        (df_with_indicators_complete["price_high"] >= trigger_bar['BUY'])
                    ):
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " BUY ORDER FILLED")
                        if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                            stop_loss = 1000
                        else:
                            stop_loss = 10
                        create_market_order_with_sl_tp(account_id, access_token, currency_pair, "BUY", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5))
                        trigger_bar = {}

                    else:
                        print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " NOTHING HAPPENS")

            time.sleep(10)

2021-08-13 19:28:21,905	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8266


In [10]:
for i in range(1, 10):
    try:
        ray.get([automated_trade.remote("EUR_USD"), automated_trade.remote("USD_JPY"),
                 automated_trade.remote("USD_CHF"), automated_trade.remote("GBP_USD"),
                 automated_trade.remote("GBP_CHF"), automated_trade.remote("GBP_JPY"),
                 automated_trade.remote("AUD_USD"), automated_trade.remote("EUR_GBP")])
    except Exception as e:
        print (e)
        print ('Restarting!')
        ray.shutdown()
        continue
    else:
        break

(pid=92087) 2021-08-13 11:25:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92084) 2021-08-13 11:20:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92083) 2021-08-13 11:25:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92082) 2021-08-13 11:25:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92086) 2021-08-13 11:25:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92079) 2021-08-13 11:20:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92081) 2021-08-13 11:25:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92085) 2021-08-13 11:20:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92084) 2021-08-13 11:25:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92079) 2021-08-13 11:25:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92085) 2021-08-13 11:25:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92087) 2021-08-13 11:30:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92081) 2021-08-13 11:30:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92082) 2021-08-13 11:30:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92086) 2021-08-13 11:30:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92084) 2021-08-13 11:30:00+00:00 EUR_GBP  NOTHING 

2021-08-13 19:55:10,363	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8266


(pid=92214) 2021-08-13 11:55:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92212) 2021-08-13 11:50:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92213) 2021-08-13 11:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92217) 2021-08-13 11:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92215) 2021-08-13 11:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92216) 2021-08-13 11:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92218) 2021-08-13 11:55:00+00:00 EUR_GBP  NOTHING HAPPENS
ray::automated_trade() (pid=92209, ip=192.168.1.103)
  File "python/ray/_raylet.pyx", line 534, in ray._raylet.execute_task
  File "/opt/anaconda3/lib/python3.8/site-packages/ray/util/tracing/tracing_helper.py", line 330, in _function_with_tracing
    return function(*args, **kwargs)
  File "<ipython-input-9-90058a948642>", line 72, in automated_trade
KeyError: 'SELL'
Restarting!


2021-08-13 20:00:06,561	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8266


(pid=92268) 2021-08-13 12:00:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92266) 2021-08-13 11:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92269) 2021-08-13 11:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92267) 2021-08-13 11:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92271) 2021-08-13 11:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92270) 2021-08-13 11:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92263) 2021-08-13 12:00:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92265) 2021-08-13 12:00:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92266) 2021-08-13 12:00:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92269) 2021-08-13 12:00:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92267) 2021-08-13 12:00:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92271) 2021-08-13 12:00:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92270) 2021-08-13 12:00:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92269) 2021-08-13 12:05:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92263) 2021-08-13 12:05:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92271) 2021-08-13 12:05:00+00:00 AUD_USD  NOTHING 

...

2021-08-13 20:55:08,831	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8266


(pid=92793) 2021-08-13 12:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92798) 2021-08-13 12:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92801) 2021-08-13 12:55:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92795) 2021-08-13 12:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92800) 2021-08-13 12:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92796) 2021-08-13 12:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92797) 2021-08-13 12:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92799) 2021-08-13 12:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92796) 2021-08-13 13:00:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92793) 2021-08-13 13:00:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92797) 2021-08-13 13:00:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92801) 2021-08-13 13:00:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92800) 2021-08-13 13:00:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92795) 2021-08-13 13:00:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92799) 2021-08-13 13:00:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92798) 2021-08-13 13:00:00+00:00 GBP_CHF  NOTHING 

2021-08-13 21:20:16,345	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8266


(pid=92966) 2021-08-13 13:15:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92967) 2021-08-13 13:15:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92960) 2021-08-13 13:20:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=92965) 2021-08-13 13:20:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=92968) 2021-08-13 13:15:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92963) 2021-08-13 13:15:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92962) 2021-08-13 13:15:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=92964) 2021-08-13 13:20:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=92966) 2021-08-13 13:20:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=92967) 2021-08-13 13:20:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=92968) 2021-08-13 13:20:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=92963) 2021-08-13 13:20:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=92962) 2021-08-13 13:20:00+00:00 EUR_GBP  NOTHING HAPPENS


KeyboardInterrupt: 

In [11]:
ray.shutdown()